# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

# Value Counts used to transform columns from str to num data
#df.home_ownership.value_counts()

# Dictionaries to convert non-numeric data in the DataFrame
own_rent_norm = {"OWN":1, "MORTGAGE":2, "RENT":3, "ANY":4}
verify_norm = {"Source Verified": 1, "Verified":1, "Not Verified":2}
issue_d_norm = {"Jan-2019":1, "Feb-2019":2, "Mar-2019":3}
pymnt_plan_norm = {"n":0}
int_status_norm = {"w":1, "f":2}
next_pay_norm = {"Apr-2019":1, "May-2019":2}
app_norm = {"Individual":1, "Joint App":2}
hardship_norm = {"N":0}
debt_settle_norm = {"N":0}

#Convert Series from String data into Ordinal
df.home_ownership = df.home_ownership.apply(lambda x: own_rent_norm[x])
df.verification_status = df.verification_status.apply(lambda x: verify_norm[x])
df.issue_d = df.issue_d.apply(lambda x: issue_d_norm[x])
df.pymnt_plan = df.pymnt_plan.apply(lambda x: pymnt_plan_norm[x])
df.initial_list_status = df.initial_list_status.apply(lambda x: int_status_norm[x])
df.next_pymnt_d = df.next_pymnt_d.apply(lambda x: next_pay_norm[x])
df.application_type = df.application_type.apply(lambda x: app_norm[x])
df.hardship_flag = df.hardship_flag.apply(lambda x: hardship_norm[x])
df.debt_settlement_flag = df.debt_settlement_flag.apply(lambda x: debt_settle_norm[x])

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,3,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,2,105000.0,1,3,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,2,56000.0,1,3,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,1,3,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,2,52000.0,2,3,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)

# Create our target
y = df.loan_status

In [6]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,2.269483,8.821371e+04,1.478007,1.726172,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.654930,1.155800e+05,0.499520,0.743862,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,1.000000,4.000000e+01,1.000000,1.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,2.000000,5.000000e+04,1.000000,1.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,2.000000,7.300000e+04,1.000000,2.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,2.000000,2.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,4.000000,8.797500e+06,2.000000,3.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [7]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [13]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score


0.7642038597659039

In [14]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   59,    33],
       [ 1932, 15181]], dtype=int64)

In [15]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.64      0.89      0.06      0.75      0.55        92
   low_risk       1.00      0.89      0.64      0.94      0.75      0.58     17113

avg / total       0.99      0.89      0.64      0.93      0.75      0.58     17205



In [16]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.08058151108532509, 'total_rec_prncp'),
 (0.06572778012765752, 'last_pymnt_amnt'),
 (0.06407649509931093, 'total_pymnt'),
 (0.05529447001150006, 'total_rec_int'),
 (0.05035857191931868, 'total_pymnt_inv'),
 (0.03701863942387551, 'issue_d'),
 (0.033190417748668315, 'int_rate'),
 (0.018914387339447853, 'dti'),
 (0.018415923975030096, 'mo_sin_old_rev_tl_op'),
 (0.01758260903669584, 'installment'),
 (0.017200081559072412, 'mths_since_recent_inq'),
 (0.015250303107920964, 'bc_open_to_buy'),
 (0.014792812186079309, 'annual_inc'),
 (0.014786724453198308, 'bc_util'),
 (0.014500012877848288, 'max_bal_bc'),
 (0.014228362194668656, 'tot_cur_bal'),
 (0.014125256880371237, 'mo_sin_old_il_acct'),
 (0.014107212891125424, 'all_util'),
 (0.014027369744551037, 'out_prncp'),
 (0.013449081384048657, 'out_prncp_inv'),
 (0.013416818447204111, 'revol_bal'),
 (0.013209265094397843, 'total_bal_ex_mort'),
 (0.013201707912875293, 'num_rev_accts'),
 (0.013121424432090776, 'total_bc_limit'),
 (0.013028688298064

### Easy Ensemble Classifier

In [17]:
# Train the Classifier
from sklearn.ensemble import GradientBoostingClassifier

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.996
Accuracy score (validation): 0.995

Learning rate:  0.1
Accuracy score (training): 0.995
Accuracy score (validation): 0.994

Learning rate:  0.25
Accuracy score (training): 0.995
Accuracy score (validation): 0.993

Learning rate:  0.5
Accuracy score (training): 0.994
Accuracy score (validation): 0.993

Learning rate:  0.75
Accuracy score (training): 0.995
Accuracy score (validation): 0.993

Learning rate:  1
Accuracy score (training): 0.996
Accuracy score (validation): 0.993



In [18]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=100,
                                        learning_rate=0.05,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).head(20)

,Prediction,Actual
0,low_risk,low_risk
1,low_risk,low_risk
2,low_risk,low_risk
3,low_risk,low_risk
4,low_risk,low_risk
5,low_risk,low_risk
6,low_risk,low_risk
7,low_risk,low_risk
8,low_risk,low_risk
9,low_risk,low_risk


In [21]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.6460379091410293

In [22]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[   27,    65],
       [   24, 17089]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.53      0.29      1.00      0.38      0.54      0.27        92
   low_risk       1.00      1.00      0.29      1.00      0.54      0.31     17113

avg / total       0.99      0.99      0.30      0.99      0.54      0.31     17205



### Final Questions

1. Which model had the best balanced accuracy score?

- The Balanced Random Forest model had higher accuracy score of 0.7642038597659039

2. Which model had the best recall score?

- The Balanced Random Froest model had a higher recall score of 0.64 for high_risk classifications

3. Which model had the best geometric mean score?

- The Balanced Random Forest model had a higher geometric mean score of 0.75

4. What are the top three features?

- The Top 3 Features by Importance:
  - Total Recieved Principal: 0.08058151108532509
  - Last Payment Amount: 0.06572778012765752
  - Toal Payment: 0.06407649509931093